# Integral Monte Carlo Computation

In [1]:
#!/usr/bin/env python
from datetime import datetime
from random import uniform
import numpy as np
import pythran

In [2]:
def integral_monte_carlo(n_total, log=False):
    # Init Monte Carlo Work
    now = datetime.now()

    # Configure Limit of Integration
    lim_x_min = 1
    lim_x_max = 4
    lim_y_min = -3
    lim_y_max = 4
    lim_z_min = -2
    lim_z_max = 2

    # Calculate part of Monte Carlo
    n_fig = 0
    for i in range(0, n_total):
        x_random = uniform(lim_x_min, lim_x_max)
        y_random = uniform(lim_y_min, lim_y_max)
        z_random = uniform(lim_z_min, lim_z_max)

        toroid = z_random ** 2 + ((x_random ** 2 + y_random ** 2) ** 0.5 - 3) ** 2
        if (x_random > 1) and (y_random >= -3) and (toroid <= 1):
            n_fig += 1

    # Calculate volume of figure
    v_total = (lim_x_max - lim_x_min) * (lim_y_max - lim_y_min) * (lim_z_max - lim_z_min)
    v_figura = round(v_total * (n_fig / float(n_total)), 2)

    # End Monte Carlo Work
    time_process = round((datetime.now() - now).total_seconds(), 2)
    
    if(log == True):
        print("> Simple Monte Carlo :: Iterations:%s :: Volume: %s :: TimeToProcess: %s" %(n_total, v_figura, time_process))
integral_monte_carlo(10000000, True)

> Simple Monte Carlo :: Iterations:10000000 :: Volume: 22.09 :: TimeToProcess: 15.22


In [3]:
%timeit integral_monte_carlo(500000)

759 ms ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%load_ext pythran.magic

In [5]:
%%pythran -fopenmp
#pythran export integral_monte_carlo_pythran_omp(int)
import numpy as np
from random import uniform
def integral_monte_carlo_pythran_omp(n_total):
    # Configure Limit of Integration
    lim_x_min = 1
    lim_x_max = 4
    lim_y_min = -3
    lim_y_max = 4
    lim_z_min = -2
    lim_z_max = 2
    
    n_fig = 0
    #omp parallel for reduction(+:n_fig)
    for i in range(0, n_total):
        x_random = uniform(lim_x_min, lim_x_max)
        y_random = uniform(lim_y_min, lim_y_max)
        z_random = uniform(lim_z_min, lim_z_max)

        toroid = z_random ** 2 + ((x_random ** 2 + y_random ** 2) ** 0.5 - 3) ** 2
        if (x_random > 1) and (y_random >= -3) and (toroid <= 1):
            n_fig += 1
    # Calculate volume of figure
    v_total = (lim_x_max - lim_x_min) * (lim_y_max - lim_y_min) * (lim_z_max - lim_z_min)
    v_figura = round(v_total * (n_fig / float(n_total)), 2)
        
    return v_figura

In [6]:
%timeit integral_monte_carlo_pythran_omp(500000)

63.9 ms ± 824 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
# Init Monte Carlo Work
now = datetime.now()

#Process Integral
integral = integral_monte_carlo_pythran_omp(10000000)

# End Monte Carlo Work
time_process = round((datetime.now() - now).total_seconds(), 2)

print("Time: %s :: Integral: %s" %(time_process, integral))

Time: 1.29 :: Integral: 22.11
